# Graph RAG

In [19]:
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_core.documents import Document


ImportError: cannot import name 'create_openai_functions_agent' from 'langchain_experimental.agents' (/Users/arslanhaider/.local/share/virtualenvs/generative-ai-XwelIRLU/lib/python3.12/site-packages/langchain_experimental/agents/__init__.py)

In [11]:
graph = Neo4jGraph(
    url="bolt://localhost:7689", username="neo4j", password="admin_neo4j"
)

In [16]:
movies_data = [
    {"title": "Inception", "director": "Christopher Nolan", "actors": ["Leonardo DiCaprio", "Tom Hardy"]},
    {"title": "Interstellar", "director": "Christopher Nolan", "actors": ["Matthew McConaughey", "Anne Hathaway"]},
    {"title": "The Dark Knight", "director": "Christopher Nolan", "actors": ["Christian Bale", "Heath Ledger"]},
    {"title": "Dunkirk", "director": "Christopher Nolan", "actors": ["Tom Hardy", "Harry Styles"]},
    {"title": "The Revenant", "director": "Alejandro González Iñárritu", "actors": ["Leonardo DiCaprio", "Tom Hardy"]},
    {"title": "Mad Max: Fury Road", "director": "George Miller", "actors": ["Tom Hardy", "Charlize Theron"]},
]

### Populate Graph

In [17]:
for movie_info in movies_data:
    title = movie_info["title"]
    director = movie_info["director"]
    actors = movie_info["actors"]

    # Create Movie node
    graph.query(f"CREATE (m:Movie {{title: '{title}'}})")

    # Create Director node and relationship
    graph.query(f"MERGE (d:Person {{name: '{director}'}}) CREATE (d)-[:DIRECTED]->(m)")

    # Create Actor nodes and relationships
    for actor in actors:
        graph.query(f"MERGE (a:Person {{name: '{actor}'}}) CREATE (a)-[:ACTED_IN]->(m)")


In [18]:
documents = [
    Document(page_content=str(movie), metadata={"title": movie["title"]})
    for movie in movies_data
]

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents, embeddings)

In [ ]:
graph_retriever = GraphRetriever(
    graph=graph,
    retrieval_query="""
    MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
    WHERE p.name = $personName
    RETURN m.title AS title
    """,
)

In [ ]:
llm = ChatOpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=vectorstore.as_retriever())

In [ ]:
def combined_retriever(query):
    graph_results = graph_retriever.get_relevant_documents(query)
    vector_results = compression_retriever.get_relevant_documents(query)
    return graph_results + vector_results

In [ ]:
qa_prompt = PromptTemplate(
    template="""Use the context below to answer the question.

    Context: {context}

    Question: {question}""",
    input_variables=["context", "question"],
)

qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),
    chain_type="stuff",
    retriever=combined_retriever,
    chain_type_kwargs={"prompt": qa_prompt},
)

In [ ]:
# Example Query 1
query = "What movies did actors who worked with Christopher Nolan also star in?"
result = qa_chain.run(query)
print(result)

#Example Query 2
query = "Tell me about the movie Inception"
result = qa_chain.run(query)
print(result)

In [ ]:
#Neo4j population example.
"""
#Neo4j population example.
#This is an example of cypher code, that would populate the graph.
#You would need to adapt it to your specific data.
#LOAD CSV WITH HEADERS FROM "file:///movies.csv" AS row
#CREATE (:Movie {title: row.title})

#LOAD CSV WITH HEADERS FROM "file:///actors.csv" AS row
#CREATE (:Person {name: row.name})

#LOAD CSV WITH HEADERS FROM "file:///directors.csv" AS row
#CREATE (:Person {name: row.name})

#LOAD CSV WITH HEADERS FROM "file:///acted_in.csv" AS row
#MATCH (p:Person {name: row.actor})
#MATCH (m:Movie {title: row.movie})
#CREATE (p)-[:ACTED_IN]->(m)

#LOAD CSV WITH HEADERS FROM "file:///directed_by.csv" AS row
#MATCH (p:Person {name: row.director})
#MATCH (m:Movie {title: row.movie})
#CREATE (p)-[:DIRECTED_BY]->(m)
"""